### Tech Exellence Advanced Data Science - Generative AI Video Classification Project

##### Project Authors: Tim Tieng, Afia Owusu-Forfie

**Objective**: Develop a model to classify video content into categories such as sports, news, movies, etc., and enhance this classification by generating descriptive captions or summaries that provide additional context about the content. This can be particularly useful for content curation platforms, accessibility applications (e.g., providing descriptions for the hearing impaired), or educational tools where supplementary information enhances learning.

**Data**: Public Dataset: k400-Dataset, which has a vast collection of labeled video data suitable for training video classification models.

In [ ]:
# Import Packages for project

# Standard Libraries
import csv
import cv2
import datetime as dt
import imageio
from IPython.display import Image
import glob
import numpy as np
import os
import pandas as pd
import re
from sys import stderr
import utils

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Algorithms, Modeling and Data Pre-processing
import feature_engine
from feature_engine.encoding import OrdinalEncoder
from feature_engine.transformation import YeoJohnsonTransformer
from scipy.stats import anderson, chi2_contingency
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,precision_score, roc_auc_score,recall_score
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Deep Learning
import keras
from keras import layers
from keras.layers import RandomFlip, RandomRotation, Rescaling, BatchNormalization, Conv2D, MaxPooling2D, Dense, Input
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Input, LSTM, Dense,GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model Optimization and Hyperparameter Tuning
import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, tpe, hp
import mlflow

import tensorboard

In [ ]:
# Obtain the Data
filepath = '../data/K400/video_annotations.csv'
raw_csv = pd.read_csv(filepath)
k400_df = pd.DataFrame(raw_csv)

k400_df.info(memory_usage='deep')

In [ ]:
# initial inspection of complete dataframe
k400_df.head

In [ ]:
# Check for null values/percentage of null values:

k400_df.isna().mean()

In [ ]:
k400_df.isna().sum()

### Observations 

1. No null values
2. Over 240k Observations
3. 6 Attributes of string/int datatypes

In [ ]:
# check for dup
num_unique = k400_df.nunique()
num_unique

### Observations

1. THere are 400 unique labels
2. There are about 20K youtube_id with only about 850 videos
3. Videos duration is only 10 seconds as annotated by the difference between time_start and time_end values

**Next Steps**: to reduce the dimensionality, I need to create a function that will map a video file to a youtube id value in the video_annotations.csv file and create a new dataframe where we have a match. Data Cleaning required on the names of the video files

In [ ]:
youtube_id_values = k400_df['youtube_id']
print(f"Total Youtube ID Values in Dataset: {youtube_id_values.count()}")
print(youtube_id_values)

In [ ]:
# Get the amount of unique youtube_id
number_unique_id = youtube_id_values.nunique()
print(f"Unique Youtube ID Values: {number_unique_id}")

In [ ]:
# Check for unique values
unique_youtube_id = youtube_id_values.unique()
unique_youtube_id

In [ ]:
# Get the number of video files we are working with
def count_video_files(directory):
    """
    Purpose - to get a video file count within a given directory
    Arguments - directory variable that holds the filepath to a video directory
    Returns - video_count of type integer
    
    """
    # Set the allowed video file extensions
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv']

    # Initialize the count
    video_count = 0

    # Iterate through all files in the directory
    for file_path in glob.glob(os.path.join(directory, '*')):
        # Check if the file has a video file extension
        if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in video_extensions):
            video_count += 1

    return video_count


In [ ]:
# Test Funcationality and return video count

# Provide the directory path to count video files
directory_path = '../data/K400/videos'

# Call the function to count video files
num_videos = count_video_files(directory_path)
print(f'Total number of video files: {num_videos} videos present')

### Video Observations

1. There seems to be a match with the youtube_id values in the video_annotations.csv file and the initial naming convention of the video files.
2. The videofile names have a timestamp that highlights how the 10second video frame was captured. 

**Next Steps**: In order to load in local video data correctly, I need to perform regular expressions to rename the video files to exclude the timestamps.

## Exploratory Data Analysis: Labels

For this project, we will explore what labels are present in the Kinetic 400 dataset

In [ ]:
k400_labels = k400_df['label']
k400_labels.tail(10)

In [ ]:
unique_labels = k400_labels.unique()
unique_labels

In [ ]:
unique_labels_count = k400_labels.nunique()
print(f'Number of Unique Video Labels: {unique_labels_count}')

### Visualize the top 50 labels in the k400 dataset

In [ ]:
# Count occurrences of each label
label_counts = k400_labels.value_counts()

# Select the top 10 labels
top_20_labels = label_counts.head(20)

# Create a countplot for the top 10 labels using Seaborn
plt.figure(figsize=(12, 8))
sns.barplot(y=top_20_labels.index, x=top_20_labels.values, palette='viridis')

# Customize plot
plt.title('Top 20 Most Frequent Labels')
plt.xlabel('Count')
plt.ylabel('Label')

# Show plot
plt.show()

### Observations

The most frequent label in the dataset is 'abseiling', which has a count close to 50. This suggests that videos of abseiling are very common in the dataset.

The activities can be categorized into several groups:

**Musical activities**: These include playing instruments like the violin, ukulele, trumpet, trombone, saxophone, recorder, piano, and organ. We can see that musical activities feature prominently in the dataset, indicating a possible focus on musical performance videos.

**Sports and physical activities**: This group includes pole vault, playing tennis, squash or racquetball, and kickball, among others. These activities are likely to involve dynamic movement, which can be useful for training algorithms to recognize physical actions.

**Recreational games**: Playing Monopoly is included, which is an indoor recreational game. This may suggest that tehre are other labels in the dataset that may represent indoor activities where movement may be minimal

### Data Preprocessing - Rename video file names for easier loading

Purpose - This step is required in order to extract features and data from the raw video files from the Kinetic Dataset. This will help later on when we split our data to feed into our future model.

In [ ]:
def remove_timestamp(filename):
    """
    Purpose: to remove the timestampe suffix at the end of our local video files
    Arguments: filename 
    Retunrs: Cleaned filename
    """
    # Split the filename by underscores
    parts = filename.rsplit('_')

    # Filter out parts that are likely numbers
    cleaned_parts = [part for part in parts if not part.isdigit()]

    # Join the cleaned parts with underscores to form the new filename
    cleaned_filename = '_'.join(cleaned_parts)

    return cleaned_filename # Remove leading/trailing whitespaces


In [ ]:
def rename_files(directory):
    """
    Purpose: To rename all the local video files in our directory for future loading 
    Arguments: Filepath to the video directory
    Returns: None

    Other Functions: Calls the remove_timestamp()
    """
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a regular file (not a directory)
        if os.path.isfile(os.path.join(directory, filename)):
            # Remove timestamp from the filename
            new_filename = remove_timestamp(filename)
            # Rename the file if the filename has changed
            if new_filename != filename:
                os.rename(os.path.join(directory, filename),
                          os.path.join(directory, new_filename))

In [ ]:
# Test
video_directory = "../data/K400/videos"

rename_files(video_directory)

In [ ]:
video_directory = '../data/K400/videos'

# Iterate through each YouTube ID
for youtube_id in youtube_id_values:
    # Find the corresponding video file in the directory
    for filename in os.listdir(video_directory):
        if youtube_id in filename:
            # Extract the file extension
            file_extension = os.path.splitext(filename)[1]

            # Construct the new file name without the timestamp
            new_filename = youtube_id + file_extension

            # Construct the full paths for old and new files
            old_filepath = os.path.join(video_directory, filename)
            new_filepath = os.path.join(video_directory, new_filename)

            # Rename the file
            os.rename(old_filepath, new_filepath)
            print(f'Renamed {filename} to {new_filename}')
            break

### Data Preparation

### Notes

train_df has 13934 Rows with 6 Attributes

test_df has 5972 rows

### Data Preprocessing - Video Data

To help with video preprocessing, I will create several functions that will help with a specific step in the process. 

**Crop_center_square()**: I begin by cropping each video frame to a centered square using my crop_center_square(frame) function to ensure uniformity in size and aspect ratio. 

**Load_video()**: I then load and preprocess the videos with my load_video(path) function, which resizes the frames to a consistent dimension and assembles them into a numpy array, capping the number at max_frames if needed. 

**Prepare_all_videos()**: Finally, I extract features from all the frames using a pre-trained InceptionV3 model in my prepare_all_videos(df, root_dir) function and create masks to accommodate sequences of varying lengths, which gives me a set of tensors ready to feed into my transformer model. 

In [ ]:
def crop_center_square(frame):
    """
    Purpose: Crop a given frame of a video into a centered squared. This will create uniformity across all the video frames available.
    Arguments:
        - frame: this is a numpy array representinga  provided video frame. Dimensions of the numpy array should be at least 2-D
    Returns: a numpy array of cropped square of the input frame
    """
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [ ]:
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def load_video2(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    if not os.path.exists(path):
        print(f"File does not exist: {path}", file=stderr)
        return np.array([])  # Return an empty array if file doesn't exist
    
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print(f"Cannot open video: {path}", file=stderr)
        return np.array([])  # Return an empty array if video cannot be opened

    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                if len(frames) == 0:
                    print(f"Failed to capture any frames from video: {path}", file=stderr)
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame.astype('float32')  # Ensure frame is in float format before preprocessing
            frame = frame[:, :, [2, 1, 0]]  # Convert from BGR to RGB
            
            # Apply model-specific preprocessing
            frame = preprocess_input(frame)

            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)



In [ ]:
def build_feature_extractor():
    """
    Purpose: Build a feature extractor model using InceptionV3 design.
    Arguments: None
    Returns: a Keras model that takes an image shape of our hyperparameters defined earlier as input and outputs a flattened feature vector


    Notes: This function initializes the InceptionV3 model with pre-trained ImageNet weights,
    excluding the top (final fully connected) layers. It modifies the network to use
    average pooling at the end and sets the expected input shape for the images. The
    output is a keras Model that takes an image input and outputs the corresponding
    feature vector. This model can be used to extract features from frames of a video
    for further analysis or processing.
    """
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(k400_df["label"])
)
print(label_processor.get_vocabulary())


In [ ]:
def prepare_all_videos(df, root_dir):
    """
    Purpose: to prepare local video data and labels for training in a sequence model (Transformers)
    Parameters:
        1. df: a pandas dataframe containing the columns youtube_id and label (From the schema of K400 Dataset)
        2. root_dir: The filepath to where the k400 datasets are stored locally
    Returns: A tuple containing two elements:  The first element is another tuple with
      two numpy arrays: frame_features of shape (num_samples, MAX_SEQ_LENGTH, NUM_FEATURES)
      and frame_masks of shape (num_samples, MAX_SEQ_LENGTH), both of which are prepared
      for the sequence model. The second element is a numpy array of processed labels.
    """
    num_samples = len(df)
    video_paths = df["youtube_id"].values.tolist()
    labels = df["label"].values
    labels = keras.ops.convert_to_numpy(label_processor(labels[..., None]))

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(
            shape=(
                1,
                MAX_SEQ_LENGTH,
            ),
            dtype="bool",
        )
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :], verbose=0,
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

In [ ]:
def prepare_all_videos2(df, root_dir):
    num_samples = len(df)
    video_paths = df['youtube_id'].values.tolist()  # Assuming these are IDs that need '.mp4' appended
    labels = df['label'].values

    MAX_SEQ_LENGTH = 10  # Example maximum sequence length
    NUM_FEATURES = 2048  # Example number of features extracted by your feature extractor

    frame_features = np.zeros((num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype='float32')
    frame_masks = np.zeros((num_samples, MAX_SEQ_LENGTH), dtype=bool)

    for idx, video_id in enumerate(video_paths):
        video_path = os.path.join(root_dir, video_id + '.mp4')  # Ensure path includes '.mp4'
        if not os.path.exists(video_path):
            print(f"File does not exist: {video_path}")  # Debugging output
            continue  # Skip this iteration if file doesn't exist

        frames = load_video2(video_path)  # Ensure your load_video2 can handle the full path with .mp4

        if frames.size == 0:
            print(f"No frames loaded from video: {video_path}")  # Additional debug info
            continue

        num_frames = frames.shape[0]
        num_frames_used = min(MAX_SEQ_LENGTH, num_frames)

        for j in range(num_frames_used):
            frame = np.expand_dims(frames[j], axis=0)  # Adding batch dimension
            features = feature_extractor.predict(frame)[0]  # Extract features
            frame_features[idx, j, :] = features
            frame_masks[idx, j] = 1  # Mark this frame as used

    return (frame_features, frame_masks), labels


In [ ]:
video_directory = '../data/K400/videos'

In [ ]:
# Testing on smaller batch of files
test_path = r"C:\Users\timot\OneDrive\Desktop\TechEx_Project_3\TEADS-Generative-AI-Video-Classification\data\K400\Test Videos"


# This could be a list you either load or define. For example:
youtube_id_values2 = ['0aAhM7nyEmM', '0BLpSquuZFM', '0TPxcKUNxbA']


# Iterate through each file in the directory
for file in os.listdir(test_path):
    # Check if the file matches any YouTube ID in your list
    for youtube_id in youtube_id_values2:
        if file.startswith(youtube_id) and not file.endswith('.mp4'):
            # Construct the full old file path
            old_file_path = os.path.join(test_path, file)
            # Add the .mp4 extension to the existing filename
            new_file_path = os.path.join(test_path, file + '.mp4')
            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed {file} to {file + '.mp4'}")
            break  # Stop checking other IDs once a match is found and renamed



In [ ]:
# Test on smalelr dataset
k400_raw = pd.read_csv("../data/K400/test_video_annotations.csv")
k400_trunc = pd.DataFrame(k400_raw)

In [ ]:
# Test on smaller set of videos
train_data3, train_labels3 = prepare_all_videos2(k400_trunc, test_path)
test_data3, test_labels3 = prepare_all_videos2(k400_trunc, test_path)

In [ ]:
print(f"Frame features in train set: {train_data3[0].shape}")
print(f"Frame masks in train set: {train_data3[1].shape}")

In [ ]:
# Convert train data
train_features3, train_masks3 = train_data3
train_features3 = np.array(train_features3)
train_masks3 = np.array(train_masks3)

# Convert test data
test_features3, test_masks3 = test_data3
test_features3 = np.array(test_features3)
test_masks3 = np.array(test_masks3)

# Convert labels
train_labels3 = np.array(train_labels3)
test_labels3 = np.array(test_labels3)

In [ ]:
# Verify
print(f"Train Features Shape: {train_features3.shape}, Type: {type(train_features3)}")
print(f"Train Masks Shape: {train_masks3.shape}, Type: {type(train_masks3)}")
print(f"Test Features Shape: {test_features3.shape}, Type: {type(test_features3)}")
print(f"Test Masks Shape: {test_masks3.shape}, Type: {type(test_masks3)}")
print(f"Train Labels Shape: {train_labels3.shape}, Type: {type(train_labels3)}")
print(f"Test Labels Shape: {test_labels3.shape}, Type: {type(test_labels3)}")

In [ ]:
# Verify the values within the train_features3 object
print(train_features3)

### Output Explanation:

The output confirms that our test_labels has data that can be used in the transformer model

### Encode Categorical Data

Encoding categorical values is crucial in video classification models to ensure compatibility with machine learning algorithms, improve model performance, and prevent biases in predictions. 
By doing this, we are able to transform the data into a format that is consumable by the model. To  accomplish this, we will instatiate an object of the LabelEncoder(), then fit_transform() the data of train_labels3

In [ ]:
# Create an encoder instance
encoder = LabelEncoder()

# Fit the encoder to the training labels, then transform them to integers
train_labels3 = encoder.fit_transform(train_labels3)

# If you have test labels, transform them using the same encoder
test_labels3 = encoder.transform(test_labels3)

# Check the transformed labels and their data type
print(train_labels3.dtype)
print(train_labels3)


**_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________**

**_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________**

### Designing Custom Transformer Model for Video Classification

In this step, we will create a custom transformer model designed for video classification. To accomplish this, we will experiment with the architecture of the layers to design a function model. 

### Create callbacks for our custom model:

For this model, we will include:

1. Tensorboard callback for experiment tracking.
2. EarlyStopping Callback to stop training when a val_loss metric has stopped improving

In [ ]:
# Create the log directory for tensorboard logs for our tensorboard callback


log_dir = "../Logs/fit/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

In [ ]:
earlystopping_callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience= 3, verbose=1)

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def build(self, input_shape):
        self.position_embeddings.build(input_shape)

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        inputs = keras.ops.cast(inputs, self.compute_dtype)
        length = keras.ops.shape(inputs)[1]
        positions = keras.ops.arange(start=0, stop=length, step=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

### PositionalEmbedding Class Explained

This code defines a custom layer in TensorFlow using the Keras API. The purpose of this layer is to add positional information to the input data, which is essential for models like Transformers that do not inherently process sequential data in order. This is further confirmed in the Keras documentation that self-attention layers that form the basic blocks of a Transformer are order-agnostic. Since videos are ordered sequences of frames, we need our Transformer model to take into account order information. We do this via positional encoding. We simply embed the positions of the frames present inside videos with an embedding layer

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation=keras.activations.gelu),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        # Debug: Ensure mask is properly broadcasted for the attention mechanism
        if mask is not None:
            # Expand mask dimensions to [batch_size, 1, 1, sequence_length]
            mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], tf.int32)
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [ ]:
def get_compiled_transformer_model(input_shape, num_classes):
    # Unpack input_shape tuple
    sequence_length, embed_dim = input_shape

    # Define the inputs
    input_features = Input(shape=input_shape, name="input_features")
    input_masks = Input(shape=(sequence_length,), name="input_masks", dtype='bool') 

    # Assuming PositionalEmbedding and TransformerEncoder are defined elsewhere
    x = PositionalEmbedding(sequence_length, embed_dim, name="frame_position_embedding")(input_features)
    x = TransformerEncoder(embed_dim, 4 * embed_dim, 1, name="transformer_layer")(x, mask=input_masks)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=[input_features, input_masks], outputs=outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
def run_experiment(train_features, train_masks, train_labels, test_features, test_masks, test_labels, num_classes):
    model = get_compiled_transformer_model(train_features.shape[1:], num_classes)  # Ensure this expects two inputs

    filepath = "video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs')
    earlystopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Ensure you pass both train_features and train_masks
    history = model.fit(
        [train_features, train_masks],  # Pass as a list if your model expects two inputs
        train_labels,
        validation_split=0.15,
        epochs=5,
        callbacks=[checkpoint, tensorboard_callback, earlystopping_callback],
        verbose=1
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate([test_features, test_masks], test_labels)  # Similarly, adjust here as needed
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

In [ ]:
# Hyperparamters 
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 5

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
# Debug
NUM_CLASSES = len(label_processor.get_vocabulary())
INPUT_SHAPE = (MAX_SEQ_LENGTH, NUM_FEATURES)

In [ ]:
test_model = get_compiled_transformer_model(INPUT_SHAPE, NUM_CLASSES)
test_model.summary()

### Model Summary Notes:

**Input Layer**: The model accepts input of shape (None, 20, 2048), where "None" can be any batch size, "20" is the sequence length, and "2048" is the feature dimension per sequence element.

**Positional Embedding Layer**: This layer adds positional information to the input, outputting the same shape (None, 20, 2048) and has 40,960 parameter to ensure the model is learning  unique embedding for each position.

**Transformer Layer**: The Transformer encoder layer maintains the shape (None, 20, 2048) with no change in sequence length or feature dimensionality, and has a 16,812,036 parameters. This layer is used for complex transformation to capture interactions between different positions in the sequence.

**Global Max Pooling Layer**: Following the Transformer layer, a GlobalMaxPooling1D layer condenses the information across the sequence length from (None, 20, 2048) to (None, 2048), taking the maximum value over the sequence for each feature.

**Dropout Layer**: A dropout layer with no change in shape (None, 2048) is used to *prevent overfitting* during training by randomly setting a portion of input units to 0 at each update during training time.

**Dense Layer**: The final output layer is a dense layer with 400 units and a softmax activation to allow the model to performing classification into 400 categories. It contains 819,680 parameters.

**Total Parameters**: The model has a total of 17,672,596 parameters, all of which are trainable. This is a large model with significant capacity for learning complex patterns in the data.

**No Non-trainable Parameters**: There are 0 non-trainable parameters, which means all the parameters in the model are being updated during training.

In [ ]:
test_model = run_experiment(
    train_features=train_features3,
    train_masks=train_masks3,
    train_labels=train_labels3,
    test_features=test_features3,
    test_masks=test_masks3,
    test_labels=test_labels3,
    num_classes=len(np.unique(train_labels3))
)

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir {log_dir}